## Демонстрация метода.

Импортируем те пакеты, которые будем использовать в работе:

In [2]:
import pandas as pd
# пандас для работы с таблицами
import nltk
# нлтк для работы с текстами (лемматизация и тд.)
import matplotlib.pyplot as plt
# матплотлиб для графиком
from sklearn.feature_extraction.text import TfidfVectorizer
# для тф-идф
from nltk.corpus import stopwords
# английские стоп-слова
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alexey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [33]:
flatten = lambda l: [item for sublist in l for item in sublist]

### 1. Подготовка данных:

Для демонстрации я буду использовать этот [датасет](https://www.kaggle.com/uciml/sms-spam-collection-dataset). 

Для начала откроем его, посмотрим на структуру данных: 

In [5]:
data = pd.read_csv('spam.csv', encoding = 'ISO-8859-1')

In [6]:
data.head(10)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
5,spam,FreeMsg Hey there darling it's been 3 week's n...,NaN,NaN,NaN
6,ham,Even my brother is not like to speak with me. ...,NaN,NaN,NaN
7,ham,As per your request 'Melle Melle (Oru Minnamin...,NaN,NaN,NaN
8,spam,WINNER!! As a valued network customer you have...,NaN,NaN,NaN
9,spam,Had your mobile 11 months or more? U R entitle...,NaN,NaN,NaN


В датасете есть лишние столбцы, которые нам не нужны -- удалим их + переименуем столбцы для удобства пользования:

In [9]:
data = data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1)

In [13]:
data = data.rename(columns={'v1': 'type', 'v2': 'message'}, index=str)

Проверим, что с данными все в порядке: 

In [17]:
print(data.shape)
data.head(10)

(5572, 2)


,type,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


### 2. Обработка сообщений:

In [19]:
from persistent.persistent import Persistent
# кастомный класс персистент
import numpy as np
# нампай для работы с коррдинатами и тд.

Теперь используя класс Persistent обработаем наши сообщения для получения persistent diagramms (предварительно их лемматизовав):

In [29]:
data.message = data.message.str.lower()

data.message = data.message.str.replace('[^\w\s]','')
# удалим лишние пробелы

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
# загрузим токенайзеры и лемматайзер

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]
#функция для передачи в пандас

data['message_l'] = data.message.apply(lemmatize_text)

Посмотрим, что получилось:

In [31]:
data.head(10)

,type,message,message_l
0,ham,go until jurong point crazy available only in ...,"[go, until, jurong, point, crazy, available, o..."
1,ham,ok lar joking wif u oni,"[ok, lar, joking, wif, u, oni]"
2,spam,free entry in 2 a wkly comp to win fa cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f..."
3,ham,u dun say so early hor u c already then say,"[u, dun, say, so, early, hor, u, c, already, t..."
4,ham,nah i dont think he goes to usf he lives aroun...,"[nah, i, dont, think, he, go, to, usf, he, lif..."
5,spam,freemsg hey there darling its been 3 weeks now...,"[freemsg, hey, there, darling, it, been, 3, we..."
6,ham,even my brother is not like to speak with me t...,"[even, my, brother, is, not, like, to, speak, ..."
7,ham,as per your request melle melle oru minnaminun...,"[a, per, your, request, melle, melle, oru, min..."
8,spam,winner as a valued network customer you have b...,"[winner, a, a, valued, network, customer, you,..."
9,spam,had your mobile 11 months or more u r entitled...,"[had, your, mobile, 11, month, or, more, u, r,..."


Теперь запустим приведение сообщений к виду персистент диаграмм:

In [34]:
texts_ph = []
# массив с текстами

for element in data.message_l:
    # цикл по лемматизированным и разделенным предложениям
    
    if element:
        # если строка не пустая и содержит слова
        a = Persistent(split_sent=element, min_count=1, window=4)
        # фигачим её в персистент класс
        dgms = a.persistent()
        # получаем диаграммы
        texts_ph.append(flatten([[p.birth for p in dgms[1]], [p.death for p in dgms[1]]]))
        # делаем просто длинный список точек как признаки
    else:
        # если пустая строка добавляем пустоту
        texts_ph.append([])

Посмотрим, как это выглядит:

In [35]:
text_ph_df = pd.DataFrame(texts_ph)
text_ph_df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,1290,1291,1292,1293,1294,1295,1296,1297,1298,1299
0,-0.002238,-0.002164,-0.001786,-0.001655,-0.001356,-0.000933,-0.000929,-0.000904,-0.000881,-0.000784,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.002216,-0.001782,-0.001640,-0.001308,-0.001286,-0.001247,-0.001158,-0.000750,-0.000748,-0.000673,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.002369,-0.002088,-0.001847,-0.001581,-0.001478,-0.001461,-0.001436,-0.001430,-0.001383,-0.001189,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.002956,-0.002011,-0.001430,-0.001012,-0.000925,-0.000760,-0.000760,-0.000711,-0.000624,-0.000564,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-0.002324,-0.001950,-0.001454,-0.001211,-0.001006,-0.000949,-0.000941,-0.000901,-0.000830,-0.000803,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,-0.002148,-0.002045,-0.001831,-0.001640,-0.001532,-0.001509,-0.001475,-0.001460,-0.001433,-0.001422,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,-0.001794,-0.001415,-0.001024,-0.001013,-0.000918,-0.000879,-0.000823,-0.000779,-0.000767,-0.000710,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,-0.002262,-0.002045,-0.001969,-0.001830,-0.001710,-0.001473,-0.001443,-0.001267,-0.001215,-0.000886,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,-0.002923,-0.002548,-0.002467,-0.002430,-0.002286,-0.002232,-0.001942,-0.001895,-0.001665,-0.001639,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,-0.002923,-0.002544,-0.001393,-0.001060,-0.001017,-0.001011,-0.000964,-0.000907,-0.000905,-0.000896,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Поскольку координаты получаются разной длины, заполним NaN-ы нулями:

In [36]:
text_ph_df = text_ph_df.fillna(0)

### 3. Обучение:

In [45]:
from sklearn.linear_model import LogisticRegression
# логистическая регрессия
from sklearn.model_selection import train_test_split
# разбиение на трейн и тест

Теперь нам надо сделать X и у: 

In [41]:
X = text_ph_df
# Матрица -- датафрейм с координатами
y = pd.factorize(data.type)[0]
# заменим слова бинарными символами

Возьмем самый простой классификатор -- логистическую регрессию: 

In [44]:
lrc = LogisticRegression(verbose=True)
# аргумент verbose -- чтобы смотреть на процесс обучения

Очень топорно разобъем признаки на трейн и тест датасеты в отношении 70/30: 

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
# разбили

Проверим, как разбили:

In [48]:
X_train.shape

(3900, 1300)

Все ок.

Зафитим модель: 

In [50]:
lrc.fit(X_train, y_train)

[LibLinear]

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=True, warm_start=False)

Сравним результаты:

* На теренеровочной выборке:

In [55]:
lrc.score(X_train, y_train)

0.86897435897435893

* На тестовой выборке:

In [56]:
lrc.score(X_test, y_test)

0.85885167464114831

Результаты на тренеровчной и тестовой выборке различаются не сильно, значит оверфита нет. Никаких фокусов и никакой магии, все работает.